# Implementation
## Paper: Measuring Semantic Similarity of Word Pairs Using Path and Information Content
#### Authors: Lingling Meng, Runqing Huang and Junzhong Gu

This is a basic implementation of above mentioned paper. Please let me know if I got something wrong in it. 
The paper is very elementary in terms of WordNet-based semantic similarity. The new measure of similarity that is proposed in the paper is a continuation of Lin-similarity measure. However it seems to perform well for certain scenarios, it does not work well globally. 


#### Dataset Format:
If the sentence pairs are to be used from a dataset file, please make sure that the dataset is in this format (or change the code relatively) 
Format: 
- SentenceA[TAB-Key]Sentence-B[TAB-Key]Known-Similarity[NextLineCharacter]
- SentenceA[TAB-Key]Sentence-B[TAB-Key]Known-Similarity[NextLineCharacter]
- SentenceA[TAB-Key]Sentence-B[TAB-Key]Known-Similarity[NextLineCharacter]
- SentenceA[TAB-Key]Sentence-B[TAB-Key]Known-Similarity[NextLineCharacter]
- ....

In [34]:
# Imports
from nltk.corpus import wordnet as wn
from math import  log
from math import exp
import os

In [35]:
# Section 2.2 - Point 6
# Node Max is the number of concept in WordNet. That is to say
# how many words are there in WordNet. For a specific version of Wordnet 
# this number is different (as more words/concepts) are added in each version. 
# For further clarification, do look at the reference [18] in paper.
node_max = 0
for s in wn.all_synsets():
    node_max = node_max + 1

In [36]:
# This is to print a few Debuggin information. 
DEBUG = False

In [37]:
# This function converts word (string format) into concepts (or say SynSets in WordNet)
# So, for example, 'dog' becomes 'dog.n.01' where it means that dog is a noun (represented by 'n')
def create_synset_from_word(word):
    return wn.synset("%s.n.01" %(str(word)))

In [38]:
# Section 2.2 - Point 1. It should be pretty clear what it means.
def lenn(synA , synB): # 'len' is a Python function so I used lenn 
    try:
        return synA.shortest_path_distance(synB, simulate_root=True)
    except:
        if DEBUG: print ("Shortest Path Distance Error...")
        return None

In [39]:
# Section 2.2 - Point3. 
#Hypernyms are 'is-a' concept in WordNet. For example, car 'is a' vechicle. 
# 'person' is a 'human'.
def depth(synA):
    try:
        return max([len(path) for path in synA.hypernym_paths()])
    except:
        if DEBUG: print ("Synset to root depth is not found...")
        return 0

In [40]:
# Section 2.2 - Point 
# The maximum depth from the certain synset (c) to its root. 
# Its like counting the distance in the graph/taxonomy.
def deep_max(synA):
    return synA.max_depth()

In [41]:
# Section 2.2 - Point 5.
# Hyponyms are also a 'is a' concept. I don't exactly know what is the difference
# difference between hypernyms and hyponyms, so please google that. 
def hypo(synA):
    return (len(synA.hyponyms()))

In [42]:
# Section 2.2 - Point 2. 
# Most specific subsumer (that connects two SynSets, i.e. c1 and c2) in the taxonomy/graph. 
# Its like the first common node between two synset nodes in the graph.
def iso(c1, c2):  #Number 2
    subsumers = c1.lowest_common_hypernyms(c2,  use_min_depth=True)
    if len(subsumers) == 0:
            return None
    subsumer = c1 if c1 in subsumers else subsumers[0]
    return subsumer


In [43]:
# Section 2.3.2 - Formula 10
# Its the Information Content based on WordNet. It assumes that the 
# Synsets which are 'leafs' in a graph (most distal nodes) represents more important 
# meaning than the one nearer to root. So for example, 'Vehicle' represents less 
# information as compare to 'car' as car is the leaf-node (distal node).
def ic(synA):
    return 1 - ( log( hypo(synA) + 1 ) ) / ( log( node_max ) )

In [44]:
# Section 3. Formula 11. 
# The new measure of similarity between two words. 
# base_ is the base in the formula 11
# power_ is the power in the formula 
# 'pow' function takes base_ and power_ and returns values as -> pow(2,3) = 2^3 = 8
# Please note that author mentions that base_ will always be between 0 >= base_ <= 1 
# (which means similarity will remain between 0 to 1) but it may not be always true. 
# Their assumption works most of the time but not always. It is not neccessary that 2*ic( iso(c1,c2) ) <= ic(c1) + ic(c2). 
# Why?, well because this assumption is based on the assumption from reference [18] which is that leaf-nodes in WordNet 
# has higher information content than the nodes nearer to 'root' which is not a proven concept but just an assumption. 
# that is why, whenever 'base_' > 1, I made it equal to 1.0 
def new_similarity_measure(c1, c2, k = 0.08, ):
    
    c1 = create_synset_from_word(c1)
    c2 = create_synset_from_word(c2)
    power_ =  ( 1 - exp ( -k*lenn(c1,c2) ) ) /  exp ( -k*lenn(c1,c2) ) 
    base_ = 2*ic( iso(c1,c2) )  / ( ic(c1) + ic(c2) ) 
    if base_ > 1:
        base_ = 1.0
#     print base_, power_, lenn(c1,c2), iso(c1,c2), ic(iso(c1,c2)), ic(c1), ic(c2)
    return pow(base_, power_)
    

In [45]:
# A simple function to process the data in file, and print it 
def process_and_find_sim():
    
    datafile = os.getcwd() +  "\\result_word_pairs2.csv"
    data = open(datafile,"r").read()
    datalines = data.split("\n")
    print "{:-^5}\t{:-^15}\t{:-^15}\t{:-^15}\t{:-^10}\t{:-^15}".format("#","Word-A","Word-B","Calc. Sim", "Meas. Sim","WUP Sim")
    
    for line in datalines[1:]:
        cols = line.split(',')
        if len(cols) == 4:
            word1, word2, groundTruth = cols[1:]
            word1, word2 = word1.lower(), word2.lower()
            syn1, syn2 = create_synset_from_word(word1), create_synset_from_word(word2)
            print "{:-^5}\t{:-^15}\t{:-^15}\t{:-^15}\t{:-^10}\t{:-^15}".format(cols[0],word1,word2,new_similarity_measure(word1,word2), float(groundTruth), syn1.wup_similarity(syn2))

In [ ]:
# Call this function. 
# process_and_find_sim()

# If you want to check some other word pairs. 
# You can do the following:
new_measure = new_similarity_measure('horse', 'bye')
print (new_measure)